<a href="https://colab.research.google.com/github/Alfikriangelo/DeepLearningTasks/blob/main/3rdWeekTask/CNN_MLP_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import library
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

In [ ]:
# Load CIFAR-10 dataset
(ds_train, ds_test), ds_info = tfds.load('cifar10', split=['train', 'test'], as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.CEQ2GA_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.CEQ2GA_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
# Preprocessing
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(128).prefetch(1)
ds_test = ds_test.map(preprocess).batch(128).prefetch(1)

In [ ]:
# CNN Model
cnn = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])


cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.fit(ds_train, epochs=30, validation_data=ds_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 22s 37ms/step - accuracy: 0.3030 - loss: 1.8808 - val_accuracy: 0.5665 - val_loss: 1.1992
Epoch 2/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5705 - loss: 1.2113 - val_accuracy: 0.6488 - val_loss: 0.9698
Epoch 3/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6435 - loss: 1.0073 - val_accuracy: 0.7062 - val_loss: 0.8442
Epoch 4/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6946 - loss: 0.8831 - val_accuracy: 0.7270 - val_loss: 0.7843
Epoch 5/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7212 - loss: 0.8014 - val_accuracy: 0.7346 - val_loss: 0.7538
Epoch 6/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7415 - loss: 0.7442 - val_accuracy: 0.7537 - val_loss: 0.7205
Epoch 7/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7602 - loss: 0.6871 - val_accuracy: 0.7589 - val_loss: 0.6925
Epoch 8/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7765 - loss: 0.6325 - val_acc

In [ ]:
# MLP Model
mlp = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(10, activation='softmax')
])

# Kompilasi model
mlp.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Scheduler agar learning rate menurun
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1
)

# Training model MLP yang dimodifikasi
mlp.fit(
    ds_train,
    epochs=30,
    validation_data=ds_test,
    callbacks=[lr_schedule]
)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.2530 - loss: 2.3140 - val_accuracy: 0.3400 - val_loss: 1.8279 - learning_rate: 0.0010
Epoch 2/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3595 - loss: 1.8011 - val_accuracy: 0.3343 - val_loss: 1.8426 - learning_rate: 0.0010
Epoch 3/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3937 - loss: 1.7000 - val_accuracy: 0.4077 - val_loss: 1.6689 - learning_rate: 0.0010
Epoch 4/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4019 - loss: 1.6750 - val_accuracy: 0.4115 - val_loss: 1.6285 - learning_rate: 0.0010
Epoch 5/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4021 - loss: 1.6647 - val_accuracy: 0.4236 - val_loss: 1.6026 - learning_rate: 0.0010
Epoch 6/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4105 - loss: 1.6550 - val_accuracy: 0.4250 - val_loss: 1.6339 - learning_rate: 0.0010
Epoch 7/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4133 - loss: 1.6425 - val_accur

In [ ]:
# Evaluation Function
def evaluate_model(model, ds):
    y_true = []
    y_pred = []
    y_score = []

    for x, y in ds:
        preds = model.predict(x)
        y_true.extend(y.numpy())
        y_pred.extend(np.argmax(preds, axis=1))
        y_score.extend(preds)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_score = np.array(y_score)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(tf.keras.utils.to_categorical(y_true), y_score, multi_class='ovo')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")

    return acc, prec, rec, f1, auc

### Evaluasi Model Deep Learning

Model deep learning seperti **CNN (Convolutional Neural Network)** dan **MLP (Multilayer Perceptron)** dievaluasi dengan berbagai metrik performa. Berikut adalah rumus-rumus matematikanya:

---

### **1. Akurasi (Accuracy)**

Akurasi mengukur proporsi prediksi yang benar dari keseluruhan data:

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

**Keterangan:**
- \( TP \): True Positive (positif yang diprediksi benar)
- \( TN \): True Negative (negatif yang diprediksi benar)
- \( FP \): False Positive (negatif yang diprediksi positif)
- \( FN \): False Negative (positif yang diprediksi negatif)

---

### **2. Presisi (Precision)**

Presisi mengukur akurasi dari prediksi positif:

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

---

### **3. Recall (Sensitivitas / True Positive Rate)**

Recall menunjukkan seberapa banyak data positif yang berhasil dideteksi oleh model:

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

---

### **4. F1 Score**

F1 Score adalah rata-rata harmonik dari presisi dan recall:

$$
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

---

### **5. False Positive Rate (FPR)**

FPR digunakan dalam kurva ROC untuk menghitung tingkat kesalahan positif:

$$
\text{FPR} = \frac{FP}{FP + TN}
$$

---

### **6. Area Under Curve (AUC - ROC)**

AUC mengukur kemampuan model dalam membedakan antara kelas positif dan negatif:

$$
\text{AUC} = \int_0^1 \text{TPR}(x) \, dx
$$

Rumus ini merepresentasikan **luas di bawah kurva ROC**, di mana:

- $$ \text{TPR}(x) $$: True Positive Rate (Recall) sebagai fungsi dari False Positive Rate

---


In [ ]:
#CNN Evaluation
print("\nEvaluasi CNN:")
evaluate_model(cnn, ds_test)


Evaluasi CNN:
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/st

(0.7964,
 0.7949452457687886,
 0.7964,
 0.7952941581235666,
 np.float64(0.9767414499999999))

In [ ]:
#MLP Evaluation
print("\nEvaluasi MLP:")
evaluate_model(mlp, ds_test)


Evaluasi MLP:
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/st

(0.5124,
 0.5104273765071072,
 0.5124000000000001,
 0.5069502202874889,
 np.float64(0.8889532444444445))

### Kesimpulan Evaluasi Model CNN dan MLP

Setelah dilakukan pelatihan dan pengujian pada dua arsitektur deep learning, yaitu **Convolutional Neural Network (CNN)** dan **Multilayer Perceptron (MLP)**, diperoleh hasil evaluasi sebagai berikut:

---

#### **1. Convolutional Neural Network (CNN)**

- **Accuracy**: 79.64%
- **Precision**: 79.49%
- **Recall**: 79.64%
- **F1 Score**: 79.53%
- **AUC (Area Under Curve)**: 97.67%

**Interpretasi:**
Model CNN menunjukkan performa yang cukup tinggi pada seluruh metrik. Akurasi dan AUC yang mendekati sempurna menunjukkan bahwa model ini sangat andal dalam membedakan antara kelas. Ini mengindikasikan bahwa CNN mampu mengenali pola dengan sangat baik, terutama untuk data visual seperti gambar.

---

#### **2. Multilayer Perceptron (MLP)**

- **Accuracy**: 51.24%
- **Precision**: 51.04%
- **Recall**: 51.24%
- **F1 Score**: 50.70%
- **AUC (Area Under Curve)**: 88.90%

**Interpretasi:**
Model MLP menunjukkan performa yang jauh lebih rendah dibandingkan CNN, terutama pada metrik akurasi dan F1 score yang hanya sedikit di atas 50%. Hal ini mengindikasikan bahwa MLP mengalami kesulitan dalam mengenali pola dari data visual yang kompleks. Meskipun demikian, nilai AUC yang cukup tinggi menunjukkan bahwa model ini masih memiliki kemampuan dalam membedakan antara kelas secara umum, meskipun klasifikasi aktualnya tidak cukup akurat. Ini memperkuat pemahaman bahwa MLP lebih cocok untuk data yang tidak memiliki struktur spasial, seperti data tabular atau vektor fitur.

---

### **Kesimpulan Umum**

- Model CNN menunjukkan performa terbaik dalam semua metrik evaluasi dan terbukti lebih efektif untuk tugas klasifikasi pada data berbasis citra.
- Model MLP memiliki keterbatasan dalam menangani data visual, namun masih menunjukkan potensi untuk digunakan pada jenis data lain yang lebih sesuai.
- Hanya CNN yang berhasil melampaui ambang minimum akurasi 75%, menjadikannya pilihan utama dalam konteks tugas klasifikasi citra ini.

---

### Rekomendasi

Untuk tugas klasifikasi yang melibatkan data visual atau spasial, CNN merupakan arsitektur yang paling tepat karena kemampuannya dalam mengekstraksi fitur spasial secara mendalam. Sebaliknya, jika data yang digunakan berupa fitur numerik atau tabular tanpa struktur spasial, MLP dapat dipertimbangkan sebagai alternatif yang lebih ringan secara komputasi.

